<a href="https://colab.research.google.com/github/obasho/obasho/blob/main/mnist%20fasion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
# Import TensorFlow Datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# Helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [2]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']
class_names = metadata.features['label'].names
print("Class names: {}".format(class_names))
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.
Class names: ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
Number of training examples: 60000
Number of test examples:     10000


In [ ]:
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

# The map function applies the normalize function to each element in the train
# and test datasets
train_dataset =  train_dataset.map(normalize)
test_dataset  =  test_dataset.map(normalize)

# The first time you use the dataset, the images will be loaded from disk
# Caching will keep them in memory, making training faster
train_dataset =  train_dataset.cache()
test_dataset  =  test_dataset.cache()

In [ ]:
inputs = tf.keras.layers.Input((28, 28, 1))
f1 = tf.keras.layers.Flatten(input_shape=(14, 14, 1))(inputs)
d1 = tf.keras.layers.Dense(128, activation=tf.nn.relu)(f1)
outputs = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(d1)

In [ ]:
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
BATCH_SIZE = 32
callbacks = [tf.keras.callbacks.EarlyStopping(patience=4, monitor='val_loss'),tf.keras.callbacks.TensorBoard(log_dir='logs'),tf.keras.callbacks.History()]
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)
history=model.fit(train_dataset, epochs=20, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE),callbacks=callbacks)

Epoch 1/20
1875/1875 [==============================] - 12s 3ms/step - loss: 0.4971 - accuracy: 0.8239
Epoch 2/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3710 - accuracy: 0.8652
Epoch 3/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3364 - accuracy: 0.8763
Epoch 4/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3145 - accuracy: 0.8850
Epoch 5/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2928 - accuracy: 0.8929
Epoch 6/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2802 - accuracy: 0.8962
Epoch 7/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2670 - accuracy: 0.9007
Epoch 8/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2552 - accuracy: 0.9054
Epoch 9/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2456 - accuracy: 0.9080
Epoch 10/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.23

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

313/313 [==============================] - 2s 4ms/step - loss: 0.3691 - accuracy: 0.8836
Accuracy on test dataset: 0.8835999965667725
